In [ ]:
'''install opencv if not already installed to use the yolo3 model'''
!pip install opencv-python==4.0.1.24

In [ ]:
import cv2, arcgis, time, imageio, tempfile, shutil, requests
import numpy as np
from arcgis.learn import YOLOv3
import pandas as pd
from PIL import Image
from io import BytesIO

In [ ]:
'''create a yolo3 model'''
yolo = YOLOv3()

In [ ]:
gis = arcgis.gis.GIS("home")

In [ ]:
'''Get your output detection layer'''
TFLTrafficCameras = gis.content.get("<PORTAL ITEM ID FOR TDL LAYER>").layers[0]
TFLTrafficCamerasDataFrame = TFLTrafficCameras.query(as_df=True)

In [ ]:
'''loop through images and get a new image for each camera'''
dirpath = tempfile.mkdtemp()
for index, row in TFLTrafficCamerasDataFrame.iterrows():
    response = requests.get(row['URL']) #request new image frame from the camera using the URL in the feature service
    if response.status_code == 200:
        image_np = np.array(Image.open(BytesIO(response.content)))
        #remove existing attachments from feature
        attachmentList = TFLTrafficCameras.attachments.get_list(row['OBJECTID'])
        if attachmentList:
            for attachment in attachmentList:
                TFLTrafficCameras.attachments.delete(row['OBJECTID'],attachment['id'])
        try:        
            predictions = yolo.predict(image_np, visualize=False, threshold=0.6)
            if predictions[0]:
                df = pd.DataFrame(np.stack((predictions[1]), axis=-1))
                featureSet = TFLTrafficCameras.query(where="OBJECTID=" + str(row['OBJECTID']))
                feature_edit = featureSet.features[0]
                feature_edit.attributes['Trucks'] = int(df[df[0] == 'truck'].count())
                feature_edit.attributes['Buses'] = int(df[df[0] == 'bus'].count())
                feature_edit.attributes['Cars'] = int(df[df[0] == 'car'].count())
                feature_edit.attributes['Pedestrians'] = int(df[df[0] == 'person'].count())
                TFLTrafficCameras.edit_features(updates=[feature_edit])

                #add original image as an attachment                           
                image_out = imageio.imwrite(dirpath + '/image.jpg', image_np)
                TFLTrafficCameras.attachments.add(row['OBJECTID'] , dirpath + '/image.jpg')

                #overlay the predictions to the orignal image
                for i in range(len(predictions[0])):
                    label = predictions[1][i]
                    if label in ['car', 'bus', 'truck','person']:
                        x1,y1,x2,y2 = (int(predictions[0][i][0]),
                               int(predictions[0][i][1]+predictions[0][i][3]),
                               int(predictions[0][i][0] + predictions[0][i][2]),
                               int(predictions[0][i][1]))
                        label = predictions[1][i]
                        if label == 'car':                          
                            cv2.rectangle(image_np,(x1-5,y1+5),(x2,y2),(0, 112, 255),3)
                            cv2.putText(image_np,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(0, 112, 255),1)
                        if label == 'bus':                          
                            cv2.rectangle(image_np,(x1-5,y1+5),(x2,y2),(255, 170, 0),3)
                            cv2.putText(image_np,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(255, 170, 0),1)
                        if label == 'truck':                          
                            cv2.rectangle(image_np,(x1-5,y1+5),(x2,y2),(197, 0, 255),3)
                            cv2.putText(image_np,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(197, 0, 255),1)
                        if label == 'person':                          
                            cv2.rectangle(image_np,(x1-5,y1+5),(x2,y2),(56, 168, 0),3)
                            cv2.putText(image_np,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(56, 168, 0),1)

                #add prediction image as an attachment 
                image_prediction = imageio.imwrite(dirpath + '/image_predictions.jpg', image_np)
                TFLTrafficCameras.attachments.add(row['OBJECTID'] , dirpath + '/image_predictions.jpg')
            else:
                #no predictions add zeros to all columns                                
                featureSet = TFLTrafficCameras.query(where="OBJECTID=" + str(row['OBJECTID']))
                feature_edit.attributes['Trucks'] = 0
                feature_edit.attributes['Buses'] = 0
                feature_edit.attributes['Cars'] = 0
                feature_edit.attributes['Pedestrians'] = 0
                TFLTrafficCameras.edit_features(updates=[feature_edit])
        except:
           print("An exception occurred")
    #if index > 10:
        #break
shutil.rmtree(dirpath)